In [1]:
from dotenv import load_dotenv
import os
from pathlib import Path

CURRENT_DIRECTORY_NOTEBOOK = None


def intitate_notebook():
    load_dotenv()
    global CURRENT_DIRECTORY_NOTEBOOK
    if CURRENT_DIRECTORY_NOTEBOOK is None:
        os.chdir(os.getenv("PROJECT_BASE_PATH"))
        CURRENT_DIRECTORY_NOTEBOOK = Path(os.getcwd())
        print("Current directory for notebook: ", CURRENT_DIRECTORY_NOTEBOOK)
    else:
        print(
            "Current directory for notebook is already set: ",
            CURRENT_DIRECTORY_NOTEBOOK,
        )

intitate_notebook()

from src.utils.common import clear_gc
clear_gc()

TRAIN_PARQUET_FOLDER_PATH = (
    "raw_main_dataset/home-credit-credit-risk-model-stability/parquet_files/train"
)

CLEANED_DATASET_DIRECTORY_PATH = Path("datasets/cleaned_data")

Current directory for notebook:  /Users/shirshmall/Personal_Drive/Credit_Risk_Model_Project


In [2]:
from src.utils.common import import_parquet_file, import_and_concatenate_parquet_files
from src.data_cleaning import get_columns_high_missing_values, get_max_per_group
from pyspark.sql.functions import countDistinct

### Data Info
- Data Name: Previous Applications (applprev_2)
- Depth: 2
- Source: Internal data source
- Relevant for Training: No - None of the columns contains any relevant information.

In [4]:
applprev2_df, spark = import_parquet_file(
    file_path=(
        CURRENT_DIRECTORY_NOTEBOOK
        / TRAIN_PARQUET_FOLDER_PATH
        / Path("depth_2/train_applprev_2.parquet")
    ),
    app_name="Previous_Application_2_Data_Cleaning_Train",
)

2024-07-25 06:58:38.008 | INFO     | src.utils.common:import_parquet_file:37 - Starting Spark session with app name: Previous_Application__Data_Cleaning_Train
2024-07-25 06:58:38.128 | INFO     | src.utils.common:import_parquet_file:55 - Reading Parquet file from path: /Users/shirshmall/Personal_Drive/Credit_Risk_Model_Project/raw_main_dataset/home-credit-credit-risk-model-stability/parquet_files/train/depth_2/train_applprev_2.parquet
2024-07-25 06:58:39.781 | INFO     | src.utils.common:import_parquet_file:57 - Parquet file loaded successfully


24/07/25 06:58:50 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
applprev2_df.show(10)

+-------+----------------------+----------------+------------------------+----------+----------+
|case_id|cacccardblochreas_147M| conts_type_509L|credacc_cards_status_52L|num_group1|num_group2|
+-------+----------------------+----------------+------------------------+----------+----------+
|      2|                  NULL|  PRIMARY_MOBILE|                    NULL|         0|         0|
|      2|                  NULL|EMPLOYMENT_PHONE|                    NULL|         0|         1|
|      2|                  NULL|  PRIMARY_MOBILE|                    NULL|         1|         0|
|      2|                  NULL|EMPLOYMENT_PHONE|                    NULL|         1|         1|
|      3|                  NULL|           PHONE|                    NULL|         0|         0|
|      3|                  NULL|  PRIMARY_MOBILE|                    NULL|         0|         1|
|      3|                  NULL|   PRIMARY_EMAIL|                    NULL|         0|         2|
|      4|                  NUL

In [7]:
missing_values_dict = get_columns_high_missing_values(df=applprev2_df, threshold=None,)

100%|██████████| 6/6 [00:01<00:00,  3.87it/s]


### Data Info
- Data Name: Person (preson_2)
- Depth: 2
- Source: Internal data source
- Relevant for Training: No - All the columns are categorical - Taking GroupBy with Max is not relevant.

In [13]:
person2_df, spark = import_parquet_file(
    file_path=(
        CURRENT_DIRECTORY_NOTEBOOK
        / TRAIN_PARQUET_FOLDER_PATH
        / Path("depth_2/train_person_2.parquet")
    ),
    app_name="Person_2_Data_Cleaning_Train",
)

2024-07-25 07:05:03.395 | INFO     | src.utils.common:import_parquet_file:37 - Starting Spark session with app name: Person_2_Data_Cleaning_Train
2024-07-25 07:05:03.833 | INFO     | src.utils.common:import_parquet_file:55 - Reading Parquet file from path: /Users/shirshmall/Personal_Drive/Credit_Risk_Model_Project/raw_main_dataset/home-credit-credit-risk-model-stability/parquet_files/train/depth_2/train_person_2.parquet
2024-07-25 07:05:03.948 | INFO     | src.utils.common:import_parquet_file:57 - Parquet file loaded successfully


In [15]:
person2_df.show()

+-------+--------------------+----------------+---------------+--------------+-----------------------+-----------------------+------------------------+----------+----------+------------------------+
|case_id|addres_district_368M|addres_role_871L|addres_zip_823M|conts_role_79M|empls_economicalst_849M|empls_employedfrom_796D|empls_employer_name_740M|num_group1|num_group2|relatedpersons_role_762T|
+-------+--------------------+----------------+---------------+--------------+-----------------------+-----------------------+------------------------+----------+----------+------------------------+
|      5|            a55475b1|            NULL|       a55475b1|      a55475b1|               a55475b1|                   NULL|                a55475b1|         0|         0|                    NULL|
|      6|          P55_110_32|         CONTACT|      P10_68_40|    P38_92_157|            P164_110_33|                   NULL|                a55475b1|         0|         0|                    NULL|
|    

In [17]:
train_credit_bureau2_df, spark = import_and_concatenate_parquet_files(
    data_file_names=[
        "train_credit_bureau_a_2_0.parquet", 
        "train_credit_bureau_a_2_1.parquet",
        "train_credit_bureau_a_2_2.parquet",
        "train_credit_bureau_a_2_3.parquet",
        "train_credit_bureau_a_2_4.parquet",
        "train_credit_bureau_a_2_5.parquet",
        "train_credit_bureau_a_2_6.parquet",
        "train_credit_bureau_a_2_7.parquet",
        "train_credit_bureau_a_2_8.parquet",
        "train_credit_bureau_a_2_9.parquet",
        "train_credit_bureau_a_2_10.parquet"
    ],
    app_name="Credit_Bureau_A_2_Train_Dataset_Cleaning",
    base_path=TRAIN_PARQUET_FOLDER_PATH / Path("depth_2"),
)

train_credit_bureau2_df.sort("case_id", "num_group1")
train_credit_bureau2_df.show(5)

2024-07-25 07:09:49.383 | INFO     | src.utils.common:import_and_concatenate_parquet_files:70 - Initializing SparkSession
2024-07-25 07:09:49.391 | INFO     | src.utils.common:import_and_concatenate_parquet_files:83 - Reading file train_credit_bureau_a_2_0.parquet with app name Credit_Bureau_A_2_Train_Dataset_Cleaning
2024-07-25 07:09:50.012 | INFO     | src.utils.common:import_and_concatenate_parquet_files:85 - File Loaded - Rows Count: 5296031
2024-07-25 07:09:50.016 | INFO     | src.utils.common:import_and_concatenate_parquet_files:83 - Reading file train_credit_bureau_a_2_1.parquet with app name Credit_Bureau_A_2_Train_Dataset_Cleaning
2024-07-25 07:09:50.258 | INFO     | src.utils.common:import_and_concatenate_parquet_files:85 - File Loaded - Rows Count: 7861809
2024-07-25 07:09:50.260 | INFO     | src.utils.common:import_and_concatenate_parquet_files:83 - Reading file train_credit_bureau_a_2_2.parquet with app name Credit_Bureau_A_2_Train_Dataset_Cleaning
2024-07-25 07:09:50.570 

+-------+-------------------------------+-------------------------------+-------------------------------+------------------------------+-------------------------------+-------------------------------+----------+----------+--------------+-------------+---------------+---------------+------------------+------------------+---------------+--------------+----------------------+----------------------+
|case_id|collater_typofvalofguarant_298M|collater_typofvalofguarant_407M|collater_valueofguarantee_1124L|collater_valueofguarantee_876L|collaterals_typeofguarante_359M|collaterals_typeofguarante_669M|num_group1|num_group2|pmts_dpd_1073P|pmts_dpd_303P|pmts_month_158T|pmts_month_706T|pmts_overdue_1140A|pmts_overdue_1152A|pmts_year_1139T|pmts_year_507T|subjectroles_name_541M|subjectroles_name_838M|
+-------+-------------------------------+-------------------------------+-------------------------------+------------------------------+-------------------------------+-------------------------------+--

In [18]:
spark.stop()